# Natural Gas Price Predictor - Example Usage

This notebook demonstrates how to use the Natural Gas Price Predictor for both price forecasting and contract valuation.

## 1. Setup

First, let's import all necessary modules:

In [ ]:
import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(''))))

from src.models.predictor import GasPricePredictor
from src.models.contract_pricer import StorageContractPricer
from src.visualization.plots import *
import pandas as pd
import matplotlib.pyplot as plt

## 2. Price Prediction

### 2.1 Initialize the Predictor

In [ ]:
predictor = GasPricePredictor('data/raw/Nat_Gas.csv')

# Get and display model metrics
metrics = predictor.get_metrics()
print("Model Performance Metrics:")
print(f"RMSE: {metrics['rmse']:.2f}")
print(f"MAE: {metrics['mae']:.2f}")
print(f"R²: {metrics['r2']:.2f}")

### 2.2 Single Date Prediction

In [ ]:
# Predict price for a specific date
future_date = '2024-12-31'
predicted_price = predictor.predict(future_date)
print(f"Predicted price for {future_date}: ${predicted_price:.2f}")

### 2.3 Multiple Date Predictions

In [ ]:
# Predict prices for multiple dates
dates = ['2024-06-30', '2024-09-30', '2024-12-31', '2025-03-31']
predictions = [(date, predictor.predict(date)) for date in dates]

# Display results
print("\nPredictions for Multiple Dates:")
for date, price in predictions:
    print(f"{date}: ${price:.2f}")

## 3. Contract Pricing

### 3.1 Initialize Contract Pricer

In [ ]:
pricer = StorageContractPricer(predictor)

### 3.2 Single Trade Contract

In [ ]:
# Calculate contract value for summer injection, winter withdrawal
result = pricer.calculate_contract_value(
    injection_dates=['2024-06-30'],
    withdrawal_dates=['2024-12-31'],
    volume_per_trade=1_000_000,
    injection_rate=50_000,
    withdrawal_rate=50_000,
    max_storage=2_000_000
)

print("\nContract Valuation Results:")
print(f"Net Contract Value: ${result['contract_value']:,.2f}")
print("\nBreakdown:")
for key, value in result['details'].items():
    if key not in ['purchase_prices', 'sale_prices']:
        print(f"{key.replace('_', ' ').title()}: ${value:,.2f}")

### 3.3 Multiple Trade Contract

In [ ]:
# Calculate value for multiple trades
result_multi = pricer.calculate_contract_value(
    injection_dates=['2024-06-30', '2024-07-31'],
    withdrawal_dates=['2024-12-31', '2025-01-31'],
    volume_per_trade=500_000,
    injection_rate=50_000,
    withdrawal_rate=50_000,
    max_storage=2_000_000
)

print("\nMultiple Trade Contract Results:")
print(f"Net Contract Value: ${result_multi['contract_value']:,.2f}")

## 4. Visualizations

### 4.1 Price History and Predictions

In [ ]:
# Load data
df = pd.read_csv('data/raw/Nat_Gas.csv')
df['Dates'] = pd.to_datetime(df['Dates'])
df.set_index('Dates', inplace=True)

# Plot price history
plot_price_history(df)

### 4.2 Seasonal Patterns

In [ ]:
plot_seasonal_patterns(df)

### 4.3 Contract Analysis

In [ ]:
plot_contract_costs(result_multi)